### 5.3 Programming Task: Digit recognition using CNNs

In [18]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from torchinfo import summary
from torch.utils.data import DataLoader


%matplotlib inline

i. Complete the code for the ConvNet class given below using the network description from supplement pdf.

In [19]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # First layer: Convolutional layer
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, stride=1)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer: 20 * 12 * 12 input features, 100 output features
        self.fc1 = nn.Linear(20 * 12 * 12, 100)
        # Final fully connected layer: 100 input features, 10 output features for class probabilities
        self.fc2 = nn.Linear(100, 10)
        
    def forward(self, x):
        # Apply convolutional layer, followed by ReLU, then max pooling
        x = self.pool(F.relu(self.conv1(x)))
        # Flatten the tensor
        x = x.view(-1, 20 * 12 * 12)
        # Apply first fully connected layer with ReLU
        x = F.relu(self.fc1(x))
        # Apply final fully connected layer
        x = self.fc2(x)
        return x



Show the net.

In [20]:
net = ConvNet()
print(net)

ConvNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2880, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


ii. Train the CNN and observe the difference in performance in comparison to the feed-forward
network from the task 5.2.

In [21]:
# Set hyper parameters.
batch_size = 64
epochs = 5
learning_rate = 0.01

In [22]:
# Load the MNIST data set.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [23]:
# Set the loss function and the optimization criteria

# Initialize the ConvNet
model = ConvNet()

# Set the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [24]:
# Run the main training loop
for epoch in range(epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/5], Step [100/938], Loss: 0.3760
Epoch [1/5], Step [200/938], Loss: 0.1212
Epoch [1/5], Step [300/938], Loss: 0.0644
Epoch [1/5], Step [400/938], Loss: 0.1728
Epoch [1/5], Step [500/938], Loss: 0.0557
Epoch [1/5], Step [600/938], Loss: 0.0801
Epoch [1/5], Step [700/938], Loss: 0.1719
Epoch [1/5], Step [800/938], Loss: 0.1763
Epoch [1/5], Step [900/938], Loss: 0.1053
Epoch [2/5], Step [100/938], Loss: 0.0483
Epoch [2/5], Step [200/938], Loss: 0.0606
Epoch [2/5], Step [300/938], Loss: 0.0834
Epoch [2/5], Step [400/938], Loss: 0.0546
Epoch [2/5], Step [500/938], Loss: 0.2247
Epoch [2/5], Step [600/938], Loss: 0.0675
Epoch [2/5], Step [700/938], Loss: 0.0689
Epoch [2/5], Step [800/938], Loss: 0.1246
Epoch [2/5], Step [900/938], Loss: 0.0110
Epoch [3/5], Step [100/938], Loss: 0.0529
Epoch [3/5], Step [200/938], Loss: 0.2956
Epoch [3/5], Step [300/938], Loss: 0.0879
Epoch [3/5], Step [400/938], Loss: 0.0076
Epoch [3/5], Step [500/938], Loss: 0.0750
Epoch [3/5], Step [600/938], Loss:

In [25]:
# Run the testing loop
model.eval()  # Set model to evaluation mode
total_correct = 0
total_samples = 0

with torch.no_grad():
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += targets.size(0)
        total_correct += (predicted == targets).sum().item()

accuracy = total_correct / total_samples * 100
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')


Accuracy of the network on the 10000 test images: 98.06%


iii. Calculate the number of learnable parameters and the output shape in each layer. Verify your
answers with model summary. (Refer last cell of the tutorial notebook)

In [26]:
# Calculate the number of learnable parameters and the output shape in each layer
print(summary(model, input_size=(1, 1, 28, 28)))


Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [1, 10]                   --
├─Conv2d: 1-1                            [1, 20, 24, 24]           520
├─MaxPool2d: 1-2                         [1, 20, 12, 12]           --
├─Linear: 1-3                            [1, 100]                  288,100
├─Linear: 1-4                            [1, 10]                   1,010
Total params: 289,630
Trainable params: 289,630
Non-trainable params: 0
Total mult-adds (M): 0.59
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 1.16
Estimated Total Size (MB): 1.25
